In [4]:
import os
from os import path
import pylangacq

In [3]:
class Paths:
  transcriptions = 'datasets/addresso/train/transcription'
  text_participants = 'datasets/addresso/train/text_participants'
  text_both = 'datasets/addresso/train/text_both'

In [38]:
def path_to_text(path, full_transcript=False):
    chat = pylangacq.read_chat(path)
    utterances = [utterance for utterance in chat.utterances()]
    tokens = [(token, utterance.participant) for utterance in utterances for token in utterance.tokens]
    words = [(token.word, participant) for (token, participant) in tokens]

    if not full_transcript:
      words = list(filter(lambda w: w[1] == 'PAR', words))
    text = ''
    for i, word_par in enumerate(words):
      word = word_par[0]
      par = word_par[1]
      if word in ['POSTCLITIC']:
        continue
      if word in ['.', '?', '!']:
        text += f'{word}\n'
      else:
        if i == 0 or words[i-1][0] in ['.', '?', '!']:
          if full_transcript:
            text += f'{par}: '
          text += word.capitalize()
        else:
          text += word
        if i < len(words) - 1 and words[i+1][0] not in ['.', ',', '?', '!']:
          text += ' '
    return text

In [32]:
print(path_to_text(path.join(Paths.transcriptions, 'S033.cha'), full_transcript=True))

INV: One more thing.
INV: What I want you to do is take a look at that picture.
PAR: Mhm.
INV: And tell me everything you see happening, everything that's going on.
PAR: Well the water's running over on the floor.
PAR: The stool is tilting.
PAR: The boy is into the cookie jar.
PAR: And his sister is reaching for a cookie.
PAR: The mother's drying dishes.
PAR: Do you want action or just want anything I see?
INV: Just action.
PAR: Okay.
INV: Okay thanks very much.
PAR: Mhm.



In [39]:
for file in os.listdir(Paths.transcriptions):
  if file.endswith('.cha'):
    ID = file.split('.')[0]

    text_participants = path_to_text(path.join(Paths.transcriptions, file), full_transcript=False)
    text_both = path_to_text(path.join(Paths.transcriptions, file), full_transcript=True)

    if not os.path.exists(Paths.text_participants):
      os.makedirs(Paths.text_participants)
    with open(path.join(Paths.text_participants, f'{ID}.txt'), 'w') as f:
      f.write(text_participants)

    if not os.path.exists(Paths.text_both):
      os.makedirs(Paths.text_both)
    with open(path.join(Paths.text_both, f'{ID}.txt'), 'w') as f:
      f.write(text_both)